# IMD3004 - IA Generativa

### Professor: Dr. Leonardo Enzo Brito da Silva

### Aluno: João Antonio Costa Paiva Chagas

# Laboratório 9: Variational Autoencoders

Código adaptado de:

[1] D. Foster, Generative Deep Learning: Teaching Machines to Paint, Write, Compose, and Play, Second Edition, O'Reilly, 2023.

[2] A. Géron, Hands-On Machine Learning with Scikit-Learning, Keras & Tensorflow, Third Edition, O'Reilly, 2022.

## Importações e configurações

In [ ]:
!pip install torchinfo

In [ ]:
from __future__ import annotations                    # Permite anotações de tipos futuristas (self-referenciais)
from tqdm.auto import tqdm                            # Barra de progresso visual para loops (compatível com notebooks e terminal)
import matplotlib.pyplot as plt                       # Biblioteca para geração de gráficos e visualizações
import numpy as np                                    # Biblioteca para operações numéricas vetorizadas
import os
import torch                                          # Biblioteca PyTorch
import torch.nn as nn                                 # Submódulo com classes de camadas neurais
import torch.nn.functional as F                       # Submódulo com funções úteis (ativação, perdas, etc.)
from scipy.stats import norm                          # Funções estatísticas (ex.: percentis e PDF da normal)
from torch.utils.data import DataLoader               # Gerenciador de mini-lotes de dados (facilita o treinamento)
from torchvision import datasets, transforms          # Conjuntos de dados e transformações de imagem do PyTorch
from torchinfo import summary                         # Sumário detalhado de modelos (tamanhos, parâmetros, shapes)
from typing import Optional                           # Para salvar os plots...

In [ ]:
# Define o dispositivo de hardware para execução: usa acelerador (GPU/TPU) se disponível; caso contrário, usa a CPU
DISPOSITIVO = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Exibe qual dispositivo está sendo utilizado
print(f"Usando {DISPOSITIVO}.")

## Armazenamento

In [ ]:
output_dir = "resultados_plots"
os.makedirs(output_dir, exist_ok=True)

## Hiperparâmetros

In [ ]:
EPOCAS_CONV            = 10                                                     # Número total de épocas de treinamento
TAMANHO_LOTE_CONV      = 128                                                    # Tamanho do mini-lote (batch size)
DIMENSAO_LATENTE_CONV  = 2                                                      # Dimensionalidade do espaço latente z

## Dados

In [ ]:
def carregar_dados_mnist(transformacao_imagem, tamanho_lote):

    # Carrega conjunto de treinamento (baixado automaticamente se não existir)
    train_data = datasets.MNIST(
        root="dados",
        train=True,
        transform=transformacao_imagem,
        download=True,
    )

    # Carrega conjunto de teste (baixado automaticamente se não existir)
    test_data = datasets.MNIST(
        root="dados",
        train=False,
        transform=transformacao_imagem,
        download=True,
    )

    # Cria DataLoaders (embaralha apenas o conjunto de treinamento)
    train_dl = DataLoader(train_data, batch_size=tamanho_lote, shuffle=True)
    test_dl = DataLoader(test_data, batch_size=tamanho_lote, shuffle=False)

    return train_dl, test_dl, len(train_data)

In [ ]:
# Define a transformação das imagens (converte para tensor normalizado em [0,1])
transformacao_densa = transforms.ToTensor()

# Carrega os conjuntos de treinamento e teste do MNIST com o tamanho de lote definido
train_dl, test_dl, _ = carregar_dados_mnist(transformacao_imagem=transformacao_densa, tamanho_lote=TAMANHO_LOTE_CONV)

## Autoencoder variacional convolucional

In [ ]:
class CamadaAmostragemLatente(nn.Module):
    """Implementa a amostragem reparametrizada z = μ + σ·ε com ε ~ N(0,I)."""

    def forward(self, media, log_variancia):
        """Gera uma amostra z a partir dos parâmetros da distribuição Gaussiana."""
        ruido_gaussiano = torch.randn_like(log_variancia)               # ε ~ N(0, 𝐈)
        desvio_padrao   = torch.exp(0.5 * log_variancia)                # σ = e^(logσ² / 2)
        vetor_latente   = media + desvio_padrao * ruido_gaussiano       # z = μ + σ·ε
        return vetor_latente

In [ ]:
class CodificadorConvolucional(nn.Module):
    """Codificador convolucional para imagens 28x28."""

    def __init__(self, dimensao_latente):
        super().__init__()

        self.bloco_convolucional = nn.Sequential(
            # Entrada: (B, 1, 28, 28)
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),               # Saída: (B, 32, 14, 14)
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),              # Saída: (B, 64, 7, 7)
            nn.ReLU(),
            nn.Flatten()                                                        # Saída: (B, 3136)
        )

        self.camada_media = nn.Linear(64 * 7 * 7, dimensao_latente)             # Camada linear para média μ
        self.camada_log_variancia = nn.Linear(64 * 7 * 7, dimensao_latente)     # Camada linear para log-variância logσ²
        self.camada_amostragem = CamadaAmostragemLatente()                      # Reparametrização: z = μ + σ·ε

    def forward(self, imagens):
        """Propaga a imagem e retorna (μ, logσ², z)."""
        x = self.bloco_convolucional(imagens)                               # Extrai e achata as características convolucionais
        media = self.camada_media(x)                                        # Vetor de médias (μ)
        log_variancia = self.camada_log_variancia(x)                        # Vetor de log-variâncias (logσ²)
        vetor_latente = self.camada_amostragem(media, log_variancia)        # Amostra reparametrizada (z = μ + σ·ε)
        return media, log_variancia, vetor_latente

In [ ]:
class DecodificadorConvolucional(nn.Module):
    """Decodificador convolucional."""

    def __init__(self, dimensao_latente):
        super().__init__()

        self.camada_densa = nn.Linear(dimensao_latente, 64 * 7 * 7)             # Mapeando o vetor latente de volta à dimensão convolucional

        self.bloco_convolucional = nn.Sequential(
            # Entrada: (B, 64, 7, 7)
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # Saída: (B, 32, 14, 14)
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),   # Saída: (B, 1, 28, 28)
            nn.Sigmoid()                                                                       # Saída ∈ (0,1)
        )

    def forward(self, vetor_latente):
        """Reconstrói a imagem a partir do vetor latente z."""
        x = self.camada_densa(vetor_latente)                                # Passa pelo primeiro bloco linear
        x = x.view(-1, 64, 7, 7)                                            # Reformata para o formato convolucional
        reconstrucao = self.bloco_convolucional(x)                          # Passa pelo pipeline convolucional
        return reconstrucao

In [ ]:
class AutoencoderVariacional(nn.Module):
    """Agrupa codificador e decodificador e calcula a função de perda MSE + β·KL."""

    def __init__(self, codificador, decodificador):
        super().__init__()
        self.codificador    = codificador                                   # Rede que gera (μ, logσ², z) a partir de x
        self.decodificador  = decodificador                                 # Rede que reconstrói x a partir de z

    def forward(self, imagens):
        """Codifica as imagens, amostra z e reconstrói as entradas."""
        media, log_variancia, vetor_latente = self.codificador(imagens)     # Passo 1: codificação x → (μ, logσ², z)
        reconstrucao = self.decodificador(vetor_latente)                    # Passo 2: decodificação (z → x̂)
        return reconstrucao, media, log_variancia

    @staticmethod
    def perda_mse_kl(reconstrucao, alvo, media, log_variancia, beta=1.0):
        """Calcula a perda total: erro de reconstrução (MSE) + regularização (β·KL)."""
        mse = F.mse_loss(reconstrucao, alvo, reduction="mean")                                  # Erro de reconstrução entre imagem original e reconstruída
        kl = -0.5 * torch.sum(1 + log_variancia - media.pow(2) - log_variancia.exp(), dim=1)    # Termo KL: mede o afastamento entre q(z|x) = N(μ,σ²) (aprendida pelo codificador) e a distribuição a priori p(z)=N(0,1) (assumida na modelagem)
        kl = kl.mean() / (alvo.size(2) * alvo.size(3))                                          # Normaliza por pixel (HxW) para manter mesma escala do MSE
        perda_total = mse + beta * kl                                                           # Combina termos: total = MSE + β·KL
        return perda_total, mse.detach(), kl.detach()                                           # Detach: impede retropropagação nos logs

## Funções auxiliares

In [ ]:
def treinar_vae(modelo, carregador_treino, epocas, otimizador, beta, nome_tag):

    tamanho_conjunto = len(carregador_treino.dataset)                                   # Número total de amostras no conjunto de treinamento
    historico_perda, historico_mse, historico_kl = [], [], []                           # Listas para armazenar o histórico de métricas por época
    progress_bar = tqdm(range(1, epocas + 1), desc=f"[{nome_tag}] Épocas", ncols=100)   # Barra de progresso visual para acompanhar o treinamento

    # Loop principal sobre as épocas
    for _ in progress_bar:
        modelo.train()                                       # Ativa modo de treinamento (dropout, BN, etc.)
        soma_perda = 0.0                                     # Acumulador de perda total
        soma_mse = 0.0                                       # Acumulador do erro de reconstrução
        soma_kl = 0.0                                        # Acumulador do termo de regularização KL

        # Loop interno sobre os mini-lotes de treinamento
        for imagens, _ in carregador_treino:
            imagens = imagens.to(DISPOSITIVO)                                                                           # Move o lote para GPU ou CPU
            otimizador.zero_grad()                                                                                      # Zera gradientes acumulados
            reconstrucao, media, log_variancia = modelo(imagens)                                                        # Forward pass: x → (μ, logσ², z) → x̂
            perda, mse, kl = AutoencoderVariacional.perda_mse_kl(reconstrucao, imagens, media, log_variancia, beta)     # Calcula perda total e seus componentes
            perda.backward()                                                                                            # Retropropaga os gradientes
            otimizador.step()                                                                                           # Atualiza os parâmetros da rede

            # Acumula perdas ponderadas pelo tamanho do mini-lote
            soma_perda += perda.item() * imagens.size(0)
            soma_mse   += mse.item() * imagens.size(0)
            soma_kl    += kl.item() * imagens.size(0)

        # Calcula médias ponderadas das métricas por época
        perda_media = soma_perda / tamanho_conjunto
        mse_media   = soma_mse / tamanho_conjunto
        kl_media    = soma_kl / tamanho_conjunto

        # Armazena histórico
        historico_perda.append(perda_media)
        historico_mse.append(mse_media)
        historico_kl.append(kl_media)

        # Atualiza a barra de progresso com as métricas atuais
        progress_bar.set_postfix({
            "Perda": f"{perda_media:.5f}",
            "MSE": f"{mse_media:.5f}",
            "KL": f"{kl_media:.5f}"
        })

    # Retorna as curvas de perda, MSE e KL por época
    return historico_perda, historico_mse, historico_kl

In [ ]:
def curvas_de_treinamento(loss_hist, mse_hist, kl_hist, save_path: Optional[str] = None):
    """Plota as curvas de perda, MSE e KL ao longo das épocas."""
    fig = plt.figure()                              # Cria a figura e define o tamanho
    plt.plot(loss_hist, label="Função de perda")    # Curva da perda total
    plt.plot(mse_hist, label="MSE")                 # Curva do erro de reconstrução
    plt.plot(kl_hist, label="KL")                   # Curva do termo de regularização
    plt.xlabel("Época")                             # Define o rótulo do eixo X
    plt.legend()                                    # Exibe a legenda das curvas
    plt.grid(True)                                  # Adiciona grade ao gráfico
    plt.tight_layout()                              # Ajusta margens automaticamente

    if save_path:
        directory = os.path.dirname(save_path)
        if directory:
            os.makedirs(directory, exist_ok=True)
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()                                      # Exibe o gráfico na tela

In [ ]:
def mostrar_reconstrucoes(modelo, carregador_teste, quantidade_imagens, save_path: Optional[str] = None):
    """Exibe imagens originais e respectivas reconstruções lado a lado."""

    modelo.eval()                                          # Coloca o modelo em modo de avaliação (desativa dropout, BN, etc.)
    imagens, _ = next(iter(carregador_teste))              # Obtém um único mini-lote do conjunto de teste
    imagens = imagens.to(DISPOSITIVO)[:quantidade_imagens] # Seleciona as N primeiras imagens e move para GPU/CPU

    with torch.no_grad():                                  # Desativa o cálculo de gradientes
        reconstrucoes, _, _ = modelo(imagens)              # Passa as imagens pelo VAE → obtém reconstruções

    fig = plt.figure(figsize=(quantidade_imagens * 1.5, 3))      # Define tamanho da figura de forma proporcional ao número de imagens

    for indice in range(quantidade_imagens):

        # Linha superior: imagens originais
        plt.subplot(2, quantidade_imagens, indice + 1)
        plt.imshow(imagens[indice].cpu().squeeze(), cmap="binary")          # Exibe imagem real (convertida para CPU)
        plt.axis("off")                                                     # Remove eixos

        # Linha inferior: imagens reconstruídas
        plt.subplot(2, quantidade_imagens, quantidade_imagens + indice + 1)
        plt.imshow(reconstrucoes[indice].cpu().squeeze(), cmap="binary")    # Exibe reconstrução correspondente
        plt.axis("off")                                                     # Remove eixos

    plt.tight_layout()                                     # Ajusta espaçamento automaticamente

    if save_path:
        directory = os.path.dirname(save_path)
        if directory:
            os.makedirs(directory, exist_ok=True)
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()                                             # Exibe a figura

In [ ]:
def visualizar_espaco_latente(modelo, carregador_dados, save_path: Optional[str] = None):
    """Projeta os vetores de médias μ no espaço latente 2D, colorindo cada ponto pela classe."""

    modelo.eval()                                         # Coloca o modelo em modo de avaliação (sem dropout, BN, etc.)
    vetores_latentes = []                                 # Lista para armazenar os vetores μ de cada amostra
    rotulos = []                                          # Lista para armazenar os rótulos correspondentes

    with torch.no_grad():                                 # Desativa o cálculo de gradientes

        for imagens, y in carregador_dados:               # Itera sobre todo o conjunto de dados
            imagens = imagens.to(DISPOSITIVO)             # Move lote para GPU/CPU
            media, _, _ = modelo.codificador(imagens)     # Extrai o vetor μ de cada imagem
            vetores_latentes.append(media.cpu().numpy())  # Move para CPU e converte em numpy
            rotulos.append(y.numpy())                     # Armazena rótulos das classes

    # Concatena todos os vetores μ e rótulos em arrays únicos
    pontos_z = np.concatenate(vetores_latentes)
    classes  = np.concatenate(rotulos)

    fig = plt.figure(figsize=(8, 6))                                                                      # Define o tamanho da figura
    grafico = plt.scatter(pontos_z[:, 0], pontos_z[:, 1], c=classes, cmap="tab10", s=10, alpha=0.7) # Cria o gráfico de dispersão dos pontos latentes coloridos por classe
    plt.colorbar(grafico, ticks=range(10), label="Classe")                                          # Adiciona barra de cores indicando as classes
    plt.title(r"Projeção de $\mu$ no espaço latente (2D)")                                          # Título do gráfico
    plt.xlabel("z[0]")                                                                              # Eixo horizontal: primeira dimensão latente
    plt.ylabel("z[1]")                                                                              # Eixo vertical: segunda dimensão latente
    plt.grid(True)                                                                                  # Adiciona grade para melhor visualização
    plt.tight_layout()                                                                              # Ajusta margens automaticamente

    if save_path:
        directory = os.path.dirname(save_path)
        if directory:
            os.makedirs(directory, exist_ok=True)
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()                                                                                      # Exibe o gráfico

In [ ]:
def visualizar_grade_latente(decodificador, tamanho_grade=20, save_path: Optional[str] = None):
    """Gera uma grade de imagens decodificadas a partir de pontos regulares no espaço latente 2D."""

    assert tamanho_grade > 1, "Tamanho da grade deve ser maior que 1."   # Garante que a grade tenha pelo menos 2 pontos por eixo

    # Gera coordenadas no espaço latente com base nos percentis da Normal padrão N(0,1)
    # Usar percentis evita pontos extremos (muito afastados da densidade central)
    eixo = norm.ppf(np.linspace(0.01, 0.99, tamanho_grade))

    linhas_imagem = []                                                   # Lista para armazenar linhas da grade de imagens
    for valor_y in eixo:                                                 # Varre o eixo vertical (dimensão z[1])
        linha_atual = []                                                 # Armazena as imagens de uma linha da grade
        for valor_x in eixo:                                             # Varre o eixo horizontal (dimensão z[0])
            # Cria o vetor latente z = [valor_x, valor_y]
            ponto_latente = torch.tensor([[valor_x, valor_y]], dtype=torch.float32, device=DISPOSITIVO)
            # Decodifica o ponto latente em uma imagem (sem gradientes)
            with torch.no_grad():
                imagem = decodificador(ponto_latente).cpu().squeeze().numpy()
            linha_atual.append(imagem)                                   # Adiciona imagem à linha atual
        # Concatena horizontalmente as imagens da linha
        linhas_imagem.append(np.concatenate(linha_atual, axis=1))

    # Concatena todas as linhas verticalmente -> imagem completa da grade
    imagem_grade = np.concatenate(linhas_imagem, axis=0)

    fig = plt.figure(figsize=(8, 8))                                           # Define tamanho da figura
    plt.imshow(imagem_grade, cmap="binary")                              # Exibe imagem composta em escala de cinza
    plt.axis("off")                                                      # Remove eixos para visualização limpa
    plt.title("Reconstruções em grade no espaço latente 2D")             # Título do gráfico
    plt.tight_layout()                                                   # Ajusta margens automaticamente
    if save_path:
        directory = os.path.dirname(save_path)
        if directory:
            os.makedirs(directory, exist_ok=True)
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()                                                                                      # Exibe o gráfico

In [ ]:
def gerar_amostras_aleatorias(modelo, dimensao_latente, total_imagens, colunas, save_path: str):
    """Gera novas amostras a partir de vetores latentes aleatórios (z ~ N(0,1))."""

    modelo.eval()                                                                           # Coloca o modelo em modo de avaliação
    with torch.no_grad():                                                                   # Desativa o cálculo de gradientes
        vetores_latentes = torch.randn(total_imagens, dimensao_latente, device=DISPOSITIVO) # Gera vetores latentes amostrados da distribuição padrão N(0,1)
        imagens = modelo.decodificador(vetores_latentes).cpu().numpy()                      # Decodifica os vetores latentes em imagens sintéticas

    linhas = (total_imagens - 1) // colunas + 1             # Calcula o número de linhas necessárias para organizar as imagens na grade
    fig = plt.figure(figsize=(colunas, linhas))                   # Define tamanho proporcional da figura

    # Exibe cada imagem na grade (linhas x colunas)
    for indice, imagem in enumerate(imagens):
        plt.subplot(linhas, colunas, indice + 1)            # Define posição do subplot
        plt.imshow(imagem.squeeze(), cmap="binary")         # Exibe imagem (remove canal e usa tons de cinza)
        plt.axis("off")                                     # Remove eixos para visualização limpa
    plt.tight_layout()                                      # Ajusta margens automaticamente
    if save_path:
        directory = os.path.dirname(save_path)
        if directory:
            os.makedirs(directory, exist_ok=True)
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()                                                                                      # Exibe o gráfico

In [ ]:
def salvar_sumario_arquitetura(model, input_size, save_path: str):
    # Guarda o resultado do summary do torchinfo
    summary_stats = summary(model, input_size=input_size, col_names=("input_size", "output_size", "num_params"), verbose=0)

    # Converte para string.
    summary_text = str(summary_stats)

    # Cria uma figura do Matplotlib para desenhar o texto
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111)
    ax.axis('off')

    # Adiciona o texto à figura
    fig.text(0.05, 0.95, summary_text, transform=fig.transFigure,
             ha="left", va="top", fontfamily='monospace', fontsize=8)

    directory = os.path.dirname(save_path)
    if directory:
        os.makedirs(directory, exist_ok=True)

    # Salva em PDF
    fig.savefig(save_path, format='pdf', bbox_inches='tight')
    plt.close(fig)
    print(f"Sumário do modelo salvo em: {save_path}")

## Experimento

In [ ]:
def executar_e_plotar_experimento(
    nome_experimento: str,
    model: nn.Module,
    output_dir: str,
    beta: float,
):

    print(f"\n==============================================")
    print(f"  Executando: {nome_experimento}")
    print(f"==============================================\n")

    otimizador_conv = torch.optim.Adam(model.parameters(), lr=1e-3)
    hist_loss, hist_mse, hist_kl = treinar_vae(
        modelo=model,
        carregador_treino=train_dl,
        epocas=EPOCAS_CONV,
        otimizador=otimizador_conv,
        beta=beta,
        nome_tag=nome_experimento,
    )

    curvas_de_treinamento(hist_loss, hist_mse, hist_kl,
                          save_path=f"{output_dir}/{nome_experimento}_curvas.pdf")

    mostrar_reconstrucoes(
        modelo=model,
        carregador_teste=test_dl,
        quantidade_imagens=10,
        save_path=f"{output_dir}/{nome_experimento}_reconstrucoes.pdf",
    )

    visualizar_espaco_latente(
        modelo=model,
        carregador_dados=test_dl,
        save_path=f"{output_dir}/{nome_experimento}_espaco_latente.pdf",
    )

    visualizar_grade_latente(
        decodificador=model.decodificador,
        tamanho_grade=25,
        save_path=f"{output_dir}/{nome_experimento}_grade_latente.pdf",
    )

    gerar_amostras_aleatorias(
        modelo=model,
        dimensao_latente=DIMENSAO_LATENTE_CONV,
        total_imagens=50,
        colunas=10,
        save_path=f"{output_dir}/{nome_experimento}_amostras.pdf",
    )



# Tarefa

- Conjunto de dados a ser utilizado MNIST (ao invés do Fashion MNIST):

    ``` python
    train_data  = datasets.MNIST(root="data", train=True, download=True)     # carrega conjunto de treinamento
    test_data   = datasets.MNIST(root="data", train=False, download=True)    # carrega conjunto de teste
    ```

1. Implemente uma versão convolucional do VAE.
2. Treine a versão convolucional do VAE com os valores diferentes para o parâmetro $\beta$ (ex.: $\beta=\in\{0, 0.1, 100\}$).
    - Mostre o termo de reconstrução (MSE), o termo de regularização (KL) e a função de perda por época (usando `curvas_de_treinamento`).
    - Mostre as reconstruções (usando `mostrar_reconstrucoes`)
    - Mostre o espaço latente (usando visualizar_espaco_latente e `visualizar_grade_latente`)
    - Gere amostras sintéticas (usando `gerar_amostras_aleatorias`)

**Entregáveis**:
1. Notebook `.ipynb`.
2. Relatório `.pdf`:
    - Reporte e comente os resultados no relatório.
    - Incluir gráficos gerados.

In [ ]:
    vae_convolucional = AutoencoderVariacional(
        CodificadorConvolucional(DIMENSAO_LATENTE_CONV),
        DecodificadorConvolucional(DIMENSAO_LATENTE_CONV),
    ).to(DISPOSITIVO)

    salvar_sumario_arquitetura(
        vae_convolucional,
        input_size=(1, 1, 28, 28),
        save_path=os.path.join(output_dir, f"conv_vae_arquitetura.pdf"),
    )



In [ ]:
for b in {0, 0.1, 100}:
    executar_e_plotar_experimento(
        nome_experimento=f"conv_vae_beta_{b}",
        model=vae_convolucional,
        output_dir=output_dir,
        beta=b
    )


In [ ]:
!zip -r /content/resultados_plots.zip /content/resultados_plots